In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet_all'

In [3]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [8]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,1):
    user_numbers = data_f['user_id'].unique()
    np.random.shuffle(user_numbers)
    dev_user_ids = user_numbers[0:531]
    validation_user_ids = user_numbers[531:581]
    exploitation_user_ids = user_numbers[581:]

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]
    exp_df = data_f.loc[data_f['user_id'].isin(exploitation_user_ids)]
    exp_vf = visual_f.loc[exp_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(1,13))]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(13,25))]
    dev_df_gen_14 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(1,15))]
    
    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(1,13))]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(13,25))]


    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel='rbf', random_state=1923)
        # clf = pipeline.Pipeline([('scaler', preprocessing.StandardScaler(with_mean=False)), ('classifier', orig_clf)])
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen_14]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.decision_function(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.decision_function(X_valid_g))




  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|#6                                                                                 | 1/50 [00:01<01:08,  1.39s/it]

  4%|###3                                                                               | 2/50 [00:02<01:09,  1.45s/it]

  6%|####9                                                                              | 3/50 [00:04<01:09,  1.47s/it]

  8%|######6                                                                            | 4/50 [00:05<01:04,  1.41s/it]

 10%|########3                                                                          | 5/50 [00:06<01:00,  1.34s/it]

 12%|#########9                                                                         | 6/50 [00:08<00:55,  1.27s/it]

 14%|###########6                                                                       | 7/50 [00:09<00:56,  1.31s/it]

 16%|#############2           

# THRESHOLD SELECTION

In [21]:
DECISION_THRESHOLD = 0.06
tn = 0
fp = 0
for val in fakes_preds:
    for pred in val:
        if pred <= DECISION_THRESHOLD: tn += 1
        else : fp += 1

print("fakes error rate:",(1-(float(tn)/(tn+fp)))*100)
        
tp = 0
fn = 0
for val in gens_preds:
    for pred in val:
        if pred >= DECISION_THRESHOLD: tp += 1
        else : fn += 1

print("gens error rate:",(1-(float(tp)/(tp+fn)))*100)

fakes error rate: 9.236947791164663
gens error rate: 9.166666666666668
